# CO Displacement Chronoamperometry
Linn Kelley<br>
08/17/2022<br>

This script calculates displaced charge for a CO displacement chronoamperometry (CA) experiment.

CO has a high affinity for Pt and will displace other adsorbed species on catalyst particles. During a displacement experiment, the cathode is held at a potential which results in either a positive or negative catalyst surface charge. This causes the adsorption of different charged species. The catalyst becomes negatively charged when held at low potentials which causes the adsorption of cations. Cations are displaced with the introduction of CO which corresponds to an oxidative current as the metal-ion bond is broken. The catalyst becomes positively charged when held at high potentials and adsorbs anions which produce a reductive current when displaced by CO. The total charge of the species displaced by CO can be found by integrating the oxidation/reduction peak associated with displacement with respect to time. This technique is primarily used to measure ionomer coverage on catalysts. Ionomer contains negatively charged sulfonate groups, so coverage is best measured at a potential where the catalyst has a positive surface charge but CO oxidation does not occurr. CO is oxidized at potentials greater than 0.4 V on Pt ionomer coverage measurements are best made at 0.4 V:<br>
<center>
$CO+\frac{1}{2}O_2\rightleftharpoons CO_2$<br>
</center>
After CO is introduced to displace surface species, it is stripped off the catalyst using cyclic voltammogram. The CO stripping charge (Q<sub>CO</sub>) and the displacement charge from the corresponding CO displacement experiment (q<sub>dis</sub>) are used to calculate sulfonate coverage according to the following equation:<br>
<center>
$\theta_{Sulfonate}=\frac{2*q_{dis}}{Q_{CO}}$
</center>
CO oxidation is a 2-electron process while CO displacement is a 1-electron process so a stoichiometric coefficient of 2 is used in the calculation.

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
%matplotlib qt

### Import file

In [2]:
disp = pd.read_csv('Sample CO displacement file.txt',delimiter='\t',header=14)

### Add a time column to the dataframe 
If your file does not have a time column, use the addtime function. 
addtime has two inputs, a dataframe (df) and the interval at which data points were collected (nu). The data collection rate can be found by opening your text file in Notepad and dividing 1/Waveform Rate (Hz).

In [3]:
def addtime(df,nu):
    time = []
    for i in range(len(df)):
        t=i*nu
        time.append(t)
    df['Time']=time 
        
addtime(disp,0.01)

### co_displacement inputs
df: dataframe with voltage, current, and time column<br>
c: color
title: title of the plot and how the image will be saved<br>
area: cell area (cm<sup>2</sup><br>)
CL: catalyst loading, mg<sub>Pt</sub>/cm<sup>2</sup><br>

The function requires user selection of the left and right side of the integration region. The user should select these points in the order:<br>
1. Left side of displacement peak
2. Right side of displacement peak

<img src="CO disp mapping points.png" width="600">

When the mapping image pops up, the magnifying glass can be used to zoom in as needed while selecting points. The home button will bring user back to the original image. Hover the mouse over the desired point and press the space bar to select a point. A red X will appear on the point. To erase the point and try again, press the backspace or delete key. When both points have been selected, press the enter key to continue. 

### co_displacement returns
q: integrated charge (mC/cm<sup>2</sup>)

### co_displacement function

In [9]:
def co_displacement(df,c,title,area):
    
    # Displacement data is plotted and user specifies integration points. 
    plt.plot(df['Time'],df['Amps']*1000/area,color=c)
    pts = plt.ginput(n=-1,timeout=-1,mouse_add=None,mouse_pop=None,mouse_stop=None)
    
    # The data indicies closest to the selected points are recorded in the variable ind.
    ind = []   
    points = list(zip(df['Time'],df['Amps']*1000/area))
    def distance(a,b):
        return(sum([(k[0]-k[1])**2 for k in zip(a,b)])**0.5)
    for i,pt in enumerate(pts):
        dists = [distance([pt[0], pt[1]],k) for k in points]
        ind.append(dists.index(min(dists)))
    
    # The time and current columns are trimmed to the integration region.
    # A line is created between the selected points.
    t = df['Time'][ind[0]:ind[1]+1]
    j = df['Amps'][ind[0]:ind[1]+1]*1000/area
    t_array = [t[ind[0]],t[ind[1]]]    
    j_array = [j[ind[0]],j[ind[1]]]
    y_line = np.interp(t,t_array,j_array)
    plt.plot(t,y_line,color=c)
    
    # The integration region is shaded and the figure is saved.
    plt.fill_between(t,y_line,j,color=c,alpha=0.5)   
    q = round((np.trapz(j,t) - np.trapz(y_line,t)),2)  
    plt.savefig(title+'.png',format='png',dpi=150)
    print(q)
    return q   

### Example use

In [10]:
q = co_displacement(disp,'mediumblue','test',50)

-2.72


In [11]:
q

-2.72

Happy integrating!